<a href="https://colab.research.google.com/github/bbenip/tetris-ai/blob/main/model/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/bbenip/tetris-ai.git

Mounted at /content/drive
Cloning into 'tetris-ai'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 125 (delta 11), reused 9 (delta 2), pack-reused 93
Receiving objects: 100% (125/125), 36.41 MiB | 14.81 MiB/s, done.
Resolving deltas: 100% (43/43), done.


Run data pre-proccessing, extracting json data for games.rar and building data.hdf5

In [49]:
!pip install tensorflow-io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.33.0
    Uninstalling tensorflow-io-gcs-filesystem-0.33.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.33.0


In [2]:
%run /content/tetris-ai/data/modify.ipynb

'Processing file: 8171/8172'

<HDF5 dataset "X_active_block": shape (2558757,), type "<i8">
<HDF5 dataset "X_block_pos": shape (2558757, 2), type "<i8">
<HDF5 dataset "X_block_rot": shape (2558757,), type "<i8">
<HDF5 dataset "X_grid": shape (2558757, 20, 10), type "|i1">
<HDF5 dataset "X_held_block": shape (2558757,), type "<i8">
<HDF5 dataset "Y": shape (2558757,), type "<i8">
<HDF5 dataset "X_active_block": shape (288380,), type "<i8">
<HDF5 dataset "X_block_pos": shape (288380, 2), type "<i8">
<HDF5 dataset "X_block_rot": shape (288380,), type "<i8">
<HDF5 dataset "X_grid": shape (288380, 20, 10), type "|i1">
<HDF5 dataset "X_held_block": shape (288380,), type "<i8">
<HDF5 dataset "Y": shape (288380,), type "<i8">
<HDF5 dataset "X_active_block": shape (2558757,), type "<i8">
<HDF5 dataset "X_block_pos": shape (2558757, 2), type "<i8">
<HDF5 dataset "X_block_rot": shape (2558757,), type "<i8">
<HDF5 dataset "X_grid": shape (2558757, 20, 10), type "|i1">
<HDF5 dataset "X_held_block": shape (2558757,), type "<i8">

Read in dataset

In [88]:
import tensorflow as tf
import tensorflow_io as tfio

DATA_DIR = "/content/tetris-ai/data"
TRAIN_DATASET_FILE_NAME = "train_data.hdf5"
TEST_DATASET_FILE_NAME = "test_data.hdf5"

train_data_file = os.path.join(DATA_DIR, TRAIN_DATASET_FILE_NAME)
test_data_file = os.path.join(DATA_DIR, TEST_DATASET_FILE_NAME)
dataset_names = ['/X_held_block', '/X_active_block', '/X_block_pos', '/X_block_rot','/X_grid','/Y']

train_data = [tfio.IODataset.from_hdf5(train_data_file, setName) for setName in dataset_names]
test_data = [tfio.IODataset.from_hdf5(test_data_file, setName)  for setName in dataset_names]

In [89]:
print(test_data)

[<HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(2,), dtype=tf.int64, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(20, 10), dtype=tf.int8, name=None)>, <HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>]


In [90]:
#import data/data.hdf5
import h5py
import numpy as np
import os
from sklearn.model_selection import train_test_split

train_held, train_active, train_pos, train_rot, train_grid, train_y = train_data
test_held, test_active, test_pos, test_rot, test_grid, test_y = test_data

In [91]:
print(train_held)

<HDF5IODataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


Convert scalar datapoints (heldBlock, activeBlock, blockRotation, nextMove) into 1-hot vector representations

e.g.

>

>Currently Held Block ⟶ **4**<br>
>Number of Block Types ⟶ **8** (including 0 for None)<br>
>1-hot vector ⟶ **[0, 0, 0, 0, 1, 0, 0, 0]**<br>



In [93]:
train_held = train_held.map(lambda x: tf.one_hot(x, 8))
train_active = train_active.map(lambda x: tf.one_hot(x, 8))
train_rot = train_rot.map(lambda x: tf.one_hot(x, 4))
train_y = train_y.map(lambda x: tf.one_hot(x, 10))

print(train_held)

<_MapDataset element_spec=TensorSpec(shape=(8,), dtype=tf.float32, name=None)>


In [94]:
test_held = test_held.map(lambda x: tf.one_hot(x, 8))
test_active = test_active.map(lambda x: tf.one_hot(x, 8))
test_rot = test_rot.map(lambda x: tf.one_hot(x, 4))
test_y =test_y.map(lambda x: tf.one_hot(x, 10))

print(test_held)

<_MapDataset element_spec=TensorSpec(shape=(8,), dtype=tf.float32, name=None)>


Train Model

In [95]:
# @title Training Constants
# @markdown Number of iterations over whole dataset
EPOCHS = 4 # @param {type:"integer"}

In [108]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, concatenate
from tensorflow.keras.models import Model

# Define input layers for each type of input
cnn_input = Input((20,10,1))
heldBlock_input = Input((8))
activeBlock_input = Input((8))
rotation_input = Input((4))
position_input = Input((2))

In [109]:
# CNN branch
x = Conv2D(32, (3, 3), activation='relu')(cnn_input)
x = Flatten()(x)

# Branch for 8-vector input
x2 = Dense(16, activation='relu')(heldBlock_input)

# Branch for 7-vector input
x3 = Dense(16, activation='relu')(activeBlock_input)

# Branch for 4-vector input
x4 = Dense(16, activation='relu')(rotation_input)

# Branch for 2-vector input
x5 = Dense(16, activation='relu')(position_input)

In [ ]:
# Concatenate all the branches
combined = concatenate([x, x2, x3, x4, x5])

In [ ]:
# Add more layers if needed
combined = Dense(64, activation='relu')(combined)
output = Dense(10, activation='softmax')(combined)  # Replace 'output_dim' with the number of output classes or neurons

In [ ]:
# Create the model
#    return (hf, [X_held_blocks, X_active_blocks, X_block_positions, X_block_rotations, X_grids, Y])
model = Model(inputs=[heldBlock_input, activeBlock_input, position_input, rotation_input, cnn_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(tf.data.Dataset.zip((train_held, train_active, train_pos, train_rot, train_grid), train_y), \
                    validation_data=tf.data.Dataset.zip((test_held, test_active, test_pos, test_rot, test_grid), test_y), \
                    epochs=EPOCHS)

In [ ]:
hf1.close()
hf2.close()

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()